In [2]:
import numpy as np
from scipy import linalg 
import matplotlib.pyplot as plt
from numpy.linalg import eig
from numpy.linalg import matrix_power
import sympy

In [27]:
from fractions import Fraction

hilb_ij = lambda i, j: Fraction(1, (i+j+1))
# def hilbert_matrix(n):
#     A = []
#     for i in range(n):
#         A.append([hilb_ij(i, j) for j in range(n)])
#     return A

hilbert_matrix = lambda n : [[hilb_ij(i,j) for j in range(n)] for i in range(n)]

A = hilbert_matrix(4)
A = sympy.Matrix(A)
A

Matrix([
[  1, 1/2, 1/3, 1/4],
[1/2, 1/3, 1/4, 1/5],
[1/3, 1/4, 1/5, 1/6],
[1/4, 1/5, 1/6, 1/7]])

In [64]:
def forward_sub(L, b, return_count=False):
    cnt = 1
    N = len(b)
    y = [0.0]*N
    y[0] = b[0] / L[0,0]
    for i in range(1, N):
        r = 0
        for j in range(i):
            r += L[i,j] * y[j]
            cnt += 2
        y[i] = (b[i] - r) / L[i,i]
        cnt += 2
    if return_count:
        return y, cnt
    return y


def backward_sub(U, y, return_count=False):
    N = len(y)
    cnt = 1
    x = [0]*N
    x[N-1] = y[N-1] / U[N-1, N-1]
    for i in range(N-2, -1, -1):
        r = 0
        for j in range(i+1, N):
            r += U[i, j] * x[j]
            cnt += 2
        x[i] = (y[i] - r) / U[i, i]
        cnt += 2
    if return_count:
        return x, cnt
    else:
        return x

def lu_decmp(B):
    N = len(B)
    # U = np.diag(np.diag(A))
    # L = np.eye(N)
    A = np.zeros((N, N))
    for j in range(N):
        for i in range(j+1):
            A[i,j] = B[i,j] - sum([A[i,k]*A[k,j] for k in range(i)])
            # print(i, j)
            # print(A)
        for i in range(j+1, N):
            A[i,j] = (B[i,j] - sum(A[i,k]*A[k,j] for k in range(j))) / A[j,j]
            # print(i, j)
            # print(A)
    U = np.triu(A)
    L = np.tril(A, -1)
    np.fill_diagonal(L, 1)
    return L, U

def solve(A, b):
    L, U = lu_decmp(A)
    y = forward_sub(L, b)
    x = backward_sub(U, y)
    return x

In [87]:
import time

In [90]:
n = 100
print("Size\tMy Time\tLAPACK")
for n in range(1, 9):
    n *= 100
    A = np.random.randn(n, n)
    K = np.array([
        [2, -1, 0, 0],
        [-1, 2, -1, 0],
        [0, -1, 2, -1],
        [0, 0, -1, 2]
    ])
    b = np.zeros(n)
    b[0] = 1

    start = time.time()
    x = solve(A, b)
    elapsed1 = time.time() - start
    start = time.time()
    x2 = linalg.solve(A, b)
    elapsed2 = time.time() - start
    print(f"{n}\t{elapsed1:.2f}\t{elapsed2:.2f}")
    if not np.allclose(x,x2):
        print("Different answers")

Size	My Time	LAPACK
100	0.12	0.00
200	0.92	0.00
300	2.91	0.09
400	6.67	0.00
500	12.82	0.00
600	22.10	0.01
700	36.49	0.01
800	52.72	0.01
